In [159]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random

from collections import OrderedDict

### Check-out flow

1. Customer approaches the register area
2. Customer queues at the register with the shortest line length
3. **Later feature**: While the customer is at the top of the stack, they can switch to another register with shorter line or fewer items, since now they have had a bit of time to survey the other registers' lines and the contents of their customers' carts
4. Each register processes customers at a rate of n seconds per item, which is a fixed cashier trait
   a. The processing rate can be increased by a constant multiplier if a bagger is added to register
   b. When register is empty (0 items), cashier can "add" a customer from a neighboring queue (FIFO)  
5. The store can open another register if it has the capacity 

In [173]:
# Person: number of items; 
# everyone has perfect vision and 
# the ability to tell which the shortest line is

class Customer:
    # TODO: change vision from [0,1] to vision radius (this relates to a store's width)
    def __init__(self, name: str, n: int, vessel: str, vision = 1):
        self.name = name
        self.items = n
        self.vision = vision
        if vessel in ["cart", "basket"]:
            self.vessel = vessel
        else:
            raise ValueError("vessel must be one of 'cart' or 'basket'")

class Cashier:
    def __init__(self, speed: int):
        """A Cashier instance

        Args:
            speed (int): number of items the cashier can process in a minute
        """
        self.speed = speed

class Register:
    
    bagger_multiplier = 1.5

    def __init__(self, index: int, cashier: Cashier, bagger: bool):
        """A Register instance

        Args:
            index (int): unique identifier
            cashier (Cashier): instance of class Cashier
            bagger (bool): whether register includes a bagger
        """
        self.index = index
        # ! New register always has 0 items, 0 customers in line, and is active
        self.items = 0
        self.line_length = 0
        self.customers = []
        self.active = True

        # ! A cashier is required when a register is opened
        self.cashier = cashier
        # A bagger is optional (you can always add one later)
        self.bagger = bagger
        # If a bagger is added to 
        if bagger:
            self.speed = cashier.speed * self.bagger_multiplier
        else:
            self.speed = cashier.speed

    def activate(self):
        self.active = True

    def deactivate(self):
        self.active = False
    
    def add_bagger(self):
    # TODO: since baggers are just normal employees, add: check Store() has enough employees free
        self.speed = self.speed * self.bagger_multiplier

    def add_customer(self, customer):
        self.line_length += 1
        self.items += customer.items
        self.customers.append(customer.name)

# TODO: class Store, with n employees, n registers, and (later) width 


# * Helper Functions 
def seq_along(list):
    return range(len(list))

def find_fewest_items(register_list: list):
    all_items = [reg.items for reg in register_list]
    min_items = [i for i in seq_along(register_list) if register_list[i].items == min(all_items)]
    return min(min_items)

def find_shortest_line(register_list: list):
    all_line_lengths = [reg.line_length for reg in registers]
    # Find register(s) with the shortest line 
    min_lines = [i for i in seq_along(registers) if registers[i].line_length == min(all_line_lengths)]
    # In case of a tie, arbitrarily pick the register with the lowest index
    # TODO: Break tie with min(items)
    return min(min_lines)

def sort_dict(dictionary):
    sorted_d = dict(sorted(dictionary.items()))
    return sorted_d

#### Build customers

In [54]:
customer1 = Customer("Miriam", random.randint(1, 100))

##### Build cashiers

In [81]:
cashier = Cashier(random.randint(1, 100))
cashier.__dict__

{'speed': 21}

#### Build registers

In [82]:
registers = []
for i in range(11):
    cashier = Cashier(random.randint(1, 100))
    current_register = Register(index=i, cashier = cashier, bagger = False)
    registers.append(current_register)

for i in [0, 2, 3, 4, 5, 9]:
    registers[i].add_customer(customer1)
for i in [5, 6, 7, 8, 9, 0]:
    registers[i].add_customer(customer1)

#### Combine Register Attributes and Weight

In [190]:
my_dict = {"width": 2, "height": 4, "depth": 5}
my_dict = sort_dict(my_dict)
attrs = ('width', 'height')
reg1_attr = [v for k,v in my_dict.items() if k in attrs]
reg1_attr

register2 = [3,4]

# ! WEIGHTS
weights = np.array([10, 20])

# ! ATTRIBUTE MATRIX
my_matrix = np.matrix([reg1_attr, register2])

# Apply customer weights to matrix
mat_weighted = np.multiply(my_matrix, weights)

# Register-wise sum
summed = np.sum(mat_weighted, axis=1)
sum_array = np.squeeze(np.asarray(summed))
names = ["reg1", "reg2"]
sum_dict = {k:v for k,v in zip(names, sum_array)}
max(sum_dict, key=sum_dict.get)

'reg2'